# Representaciones Tradicionales

En este notebook se generan representaciones tradicionales de texto para las noticias ya resumidas. Se aplicarán dos técnicas principales: Bag of Words (BoW) y TF-IDF.

Importar todas las librerías necesarias

In [1]:
import pandas as pd
import os
import string
import spacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pickle

Definir directorios de donde queremos cargar los datos resumidos y donde queremos guardar las representaciones tradicionales

In [2]:
# Usa el directorio actual del notebook como raíz del proyecto
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Define las rutas a los directorios de datos
SUMMARIES_DIR = os.path.join(ROOT_DIR, 'data', 'news', 'summarized')
OUTPUT_DIR = os.path.join(ROOT_DIR, 'data', 'news', 'traditional')

# Crea el directorio de salida si no existe
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Lista todos los archivos .parquet en el directorio SUMMARIES_DIR
parquet_files = [f for f in os.listdir(SUMMARIES_DIR) if f.endswith('.parquet')]

# Muestra el número total de archivos encontrados
print(f"Se han encontrado {len(parquet_files)} archivos parquet para procesar.\n")

# Lista los nombres de los archivos encontrados
for file in parquet_files:
    print(file)

Se han encontrado 7 archivos parquet para procesar.

amazon_news.parquet
apple_news.parquet
google_news.parquet
meta_news.parquet
microsoft_news.parquet
nvidia_news.parquet
tesla_news.parquet


Cargar el modelo de spaCy para procesamiento de lenguaje natural

In [3]:
print("Cargando modelo de spaCy...")
nlp = spacy.load("en_core_web_lg")  # Para instalar: python -m spacy download en_core_web_lg
print("Modelo de spaCy cargado.")

Cargando modelo de spaCy...
Modelo de spaCy cargado.
Modelo de spaCy cargado.


Definir funciones de preprocesamiento de texto: limpieza básica (minúsculas, eliminación de puntuación y números) y preprocesamiento completo (lematización y eliminación de stopwords)

In [4]:
def clean_text(text):
    """
    Limpia el texto: minúsculas, eliminar puntuación y números.
    """
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = ''.join([c for c in text if not c.isdigit()])
    return text

def preprocess_text(text):
    """
    Limpia y lematiza el texto, eliminando stopwords.
    """
    text = clean_text(text)
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.lemma_ != '-PRON-']
    return " ".join(tokens)

Definir el tamaño de lote para procesamiento en batches

In [5]:
BATCH_SIZE = 500

Procesamiento de archivos: para cada archivo se realiza la carga de datos, concatenación de título y resumen, preprocesamiento del texto, generación de representaciones Bag of Words y TF-IDF, y guardado de resultados junto con los vectorizadores

In [6]:
for file_idx, file_name in enumerate(parquet_files, 1):
    print(f"[{file_idx}/{len(parquet_files)}] Procesando {file_name}...")
    
    # Cargar dataset
    df = pd.read_parquet(os.path.join(SUMMARIES_DIR, file_name), engine="pyarrow")
    
    # Concatenar título y resumen
    df["text"] = df["title"] + "\n\n" + df["body_summary"]
    df = df.drop(columns=["clean_body", "body_summary", "teaser", "title", "author"])
    
    # Eliminar filas con texto vacío
    original_count = len(df)
    df = df.dropna(subset=['text'])
    final_count = len(df)
    
    if original_count != final_count:
        print(f"  Se han eliminado {original_count - final_count} filas con texto vacío")
    
    print(f"  Preprocesando {final_count} textos en lotes de {BATCH_SIZE}...")
    
    # Aplicar preprocesamiento en batches
    processed_texts = []
    num_batches = (final_count + BATCH_SIZE - 1) // BATCH_SIZE
    
    for batch_idx in range(num_batches):
        start_idx = batch_idx * BATCH_SIZE
        end_idx = min((batch_idx + 1) * BATCH_SIZE, final_count)
        
        batch_texts = df['text'].iloc[start_idx:end_idx]
        
        for idx, text in enumerate(batch_texts, start=start_idx + 1):
            if idx % 100 == 0 or idx == final_count:
                print(f"    Progreso: {idx}/{final_count} ({100*idx/final_count:.1f}%)", end='\r')
            processed_texts.append(preprocess_text(text))
    
    df['text_processed'] = processed_texts
    print()  # Nueva línea después del progreso
    
    # Generar Bag of Words
    print("  Generando representación Bag of Words...")
    bow_vectorizer = CountVectorizer(max_features=5000)
    
    # Entrenar vectorizador con todos los datos
    bow_vectorizer.fit(df['text_processed'])
    
    # Transformar en batches
    bow_vectors = []
    for batch_idx in range(num_batches):
        start_idx = batch_idx * BATCH_SIZE
        end_idx = min((batch_idx + 1) * BATCH_SIZE, final_count)
        
        batch_processed = df['text_processed'].iloc[start_idx:end_idx]
        batch_bow = bow_vectorizer.transform(batch_processed)
        bow_vectors.extend(batch_bow.toarray())
        
        if (batch_idx + 1) % 5 == 0 or (batch_idx + 1) == num_batches:
            print(f"    Lote {batch_idx + 1}/{num_batches} procesado", end='\r')
    
    df['bow'] = bow_vectors
    print()  # Nueva línea
    
    # Generar TF-IDF
    print("  Generando representación TF-IDF...")
    tfidf_vectorizer = TfidfVectorizer(max_features=5000)
    
    # Entrenar vectorizador con todos los datos
    tfidf_vectorizer.fit(df['text_processed'])
    
    # Transformar en batches
    tfidf_vectors = []
    for batch_idx in range(num_batches):
        start_idx = batch_idx * BATCH_SIZE
        end_idx = min((batch_idx + 1) * BATCH_SIZE, final_count)
        
        batch_processed = df['text_processed'].iloc[start_idx:end_idx]
        batch_tfidf = tfidf_vectorizer.transform(batch_processed)
        tfidf_vectors.extend(batch_tfidf.toarray())
        
        if (batch_idx + 1) % 5 == 0 or (batch_idx + 1) == num_batches:
            print(f"    Lote {batch_idx + 1}/{num_batches} procesado", end='\r')
    
    df['tfidf'] = tfidf_vectors
    print()  # Nueva línea
    
    # Eliminar columna de texto preprocesado
    df = df.drop(columns=['text_processed'])
    
    # Guardar en directorio de salida
    base_name = file_name.replace('_news.parquet', '')
    output_name = f"{base_name}_traditional.parquet"
    output_path = os.path.join(OUTPUT_DIR, output_name)
    df.to_parquet(output_path, engine="pyarrow", index=False)
    
    # Guardar vectorizadores para uso futuro
    bow_vectorizer_path = os.path.join(OUTPUT_DIR, f"{base_name}_bow_vectorizer.pkl")
    tfidf_vectorizer_path = os.path.join(OUTPUT_DIR, f"{base_name}_tfidf_vectorizer.pkl")
    
    with open(bow_vectorizer_path, 'wb') as f:
        pickle.dump(bow_vectorizer, f)
    with open(tfidf_vectorizer_path, 'wb') as f:
        pickle.dump(tfidf_vectorizer, f)
    
    bow_dim = len(bow_vectors[0])
    tfidf_dim = len(tfidf_vectors[0])
    print(f"  Guardado {output_name} con {len(df)} registros")
    print(f"    Dimensión BoW: {bow_dim}, Dimensión TF-IDF: {tfidf_dim}\n")

print("Todos los archivos han sido procesados exitosamente!")
print("Archivos guardados en:", OUTPUT_DIR)

[1/7] Procesando amazon_news.parquet...
  Preprocesando 12391 textos en lotes de 500...
    Progreso: 12391/12391 (100.0%)
  Generando representación Bag of Words...
    Progreso: 12391/12391 (100.0%)
  Generando representación Bag of Words...
    Lote 25/25 procesado
  Generando representación TF-IDF...
    Lote 25/25 procesado
  Generando representación TF-IDF...
    Lote 25/25 procesado
    Lote 25/25 procesado
  Guardado amazon_traditional.parquet con 12391 registros
    Dimensión BoW: 5000, Dimensión TF-IDF: 5000

[2/7] Procesando apple_news.parquet...
  Guardado amazon_traditional.parquet con 12391 registros
    Dimensión BoW: 5000, Dimensión TF-IDF: 5000

[2/7] Procesando apple_news.parquet...
  Preprocesando 24208 textos en lotes de 500...
  Preprocesando 24208 textos en lotes de 500...
    Progreso: 24208/24208 (100.0%)
  Generando representación Bag of Words...
    Progreso: 24208/24208 (100.0%)
  Generando representación Bag of Words...
    Lote 49/49 procesado
  Generando r